Вывод пройденного расстояния за период (в разбивке по
дням);

- Вывод количества выполненных заказов за период (в
разбивке по дням);
- Время проведенное в движении за период (в разбивке по
дням);
- Время простоя за период (в разбивке по дням);
- Время нахождения в каждом из статусов за период.


In [1]:
import pandas as pd
from clickhouse_driver import Client


In [2]:
client = Client(host='fake', port=9035, user='fake', password='fake', database='fake')


In [3]:
# parametered SQL query to fetch data
params = {
'forklit' : [0, 1],
'warehouse' :  [0, 1],
# _ts = YYYY-MM-DD HH:mm
'from_ts' : "2023-10-21 12:42:00.000",
'to_ts' : "2023-10-21 12:45:00.000",
}
query = f'''
SELECT *
FROM `default`.main_data_stg_t
WHERE id_forklift in %(forklit)s 
and id_warehouse in  %(warehouse)s
and event_timestamp BETWEEN %(from_ts)s and %(to_ts)s
ORDER by event_timestamp ASC ;
'''

In [4]:
result = client.execute(query, params=params)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in client.execute("DESCRIBE `default`.main_data_stg_t")])


In [5]:
# configs.py file
# Справочник маршрутов погрузчиков по складу.
# Каждый маршрут имеет id, id целевого стеллажа, а также набор контрольных точек.
# Каждая контрольная точка имеет свой id, имя, а также расстояние до следующей КТ
path_lib = {
    1: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 10},
            {"check_point_id": 3,
             "check_point_name": "K3",
             "next_check_point_distance": 10}
        ],
        "target_rack_id": "X1"},
    2: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 10},
            {"check_point_id": 3,
             "check_point_name": "K3",
             "next_check_point_distance": 15},
            {"check_point_id": 4,
             "check_point_name": "K4",
             "next_check_point_distance": 10}
        ],
        "target_rack_id": "X2"},
    3: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 5},
            {"check_point_id": 5,
             "check_point_name": "K5",
             "next_check_point_distance": 10},
            {"check_point_id": 6,
             "check_point_name": "K6",
             "next_check_point_distance": 10}
        ],
        "target_rack_id": "X3"},
    4: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 5},
            {"check_point_id": 5,
             "check_point_name": "K5",
             "next_check_point_distance": 10},
            {"check_point_id": 6,
             "check_point_name": "K6",
             "next_check_point_distance": 15},
            {"check_point_id": 7,
             "check_point_name": "K7",
             "next_check_point_distance": 10}
        ],
        "target_rack_id": "X4"},
    5: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 5},
            {"check_point_id": 5,
             "check_point_name": "K5",
             "next_check_point_distance": 5},
            {"check_point_id": 8,
             "check_point_name": "K8",
             "next_check_point_distance": 10},
            {"check_point_id": 9,
             "check_point_name": "K9",
             "next_check_point_distance": 5}
        ],
        "target_rack_id": "X5"},
    6: {"path_sequence": [
            {"check_point_id": 1,
             "check_point_name": "K1",
             "next_check_point_distance": 5},
            {"check_point_id": 2,
             "check_point_name": "K2",
             "next_check_point_distance": 5},
            {"check_point_id": 5,
             "check_point_name": "K5",
             "next_check_point_distance": 5},
            {"check_point_id": 8,
             "check_point_name": "K8",
             "next_check_point_distance": 10},
            {"check_point_id": 9,
             "check_point_name": "K9",
             "next_check_point_distance": 15},
            {"check_point_id": 10,
             "check_point_name": "K10",
             "next_check_point_distance": 10}
        ],
        "target_rack_id": "X6"}
}

# cities list
cities = [
    "Rostov-on-Don",
    "Moscow",
    "Krasnodar",
    "Tver",
    "Saint Petersburg",
    "Almaty"
]


In [83]:
# Generate a map of dists 
dists = {}
for id, path in path_lib.items():
    curr_pos =  path['path_sequence'][0]['check_point_name']
    next_dist = path['path_sequence'][0]['next_check_point_distance']
    for next_detail in path['path_sequence'][1:]:
        next_pos = next_detail['check_point_name']
        if curr_pos not in dists:
            dists[curr_pos] = {curr_pos:0}
        if next_pos not in dists:
            dists[next_pos] = {next_pos:0}
        dists[curr_pos][next_pos] = next_dist
        dists[next_pos][curr_pos] = next_dist
        curr_pos = next_pos
        next_dist = next_detail['next_check_point_distance']
    if curr_pos not in dists:
        dists[curr_pos] = {curr_pos:0}
    if path["target_rack_id"] not in dists:
        dists[path["target_rack_id"]] = {path["target_rack_id"]:0}
    dists[curr_pos][path["target_rack_id"]] = next_dist
    dists[path["target_rack_id"]][curr_pos] = next_dist

dists

{'K1': {'K1': 0, 'K2': 5},
 'K2': {'K2': 0, 'K1': 5, 'K3': 10, 'K5': 5},
 'K3': {'K3': 0, 'K2': 10, 'X1': 10, 'K4': 15},
 'X1': {'X1': 0, 'K3': 10},
 'K4': {'K4': 0, 'K3': 15, 'X2': 10},
 'X2': {'X2': 0, 'K4': 10},
 'K5': {'K5': 0, 'K2': 5, 'K6': 10, 'K8': 5},
 'K6': {'K6': 0, 'K5': 10, 'X3': 10, 'K7': 15},
 'X3': {'X3': 0, 'K6': 10},
 'K7': {'K7': 0, 'K6': 15, 'X4': 10},
 'X4': {'X4': 0, 'K7': 10},
 'K8': {'K8': 0, 'K5': 5, 'K9': 10},
 'K9': {'K9': 0, 'K8': 10, 'X5': 5, 'K10': 15},
 'X5': {'X5': 0, 'K9': 5},
 'K10': {'K10': 0, 'K9': 15, 'X6': 10},
 'X6': {'X6': 0, 'K10': 10}}

In [7]:
# Вывод количества выполненных заказов за период (в разбивке по дням)

# parametered SQL query to fetch data
params = {
'forklit' : [0, 1],
'warehouse' :  [0, 1],
# _ts = YYYY-MM-DD HH:mm
'from_ts' : "2023-10-21 12:42:00.000",
'to_ts' : "2023-10-21 12:45:00.000",
}
query = f'''
SELECT COUNT(*)
FROM `default`.main_data_stg_t
WHERE id_forklift in %(forklit)s 
and id_warehouse in  %(warehouse)s
and event_timestamp BETWEEN %(from_ts)s and %(to_ts)s
and status = "FINISH"
ORDER by event_timestamp ASC ;
'''

In [21]:
# Время простоя за период (в разбивке по дням);
# Время проведенное в движении за период (в разбивке по дням);
# parametered SQL query to fetch data
params = {
'forklit' : [0],
'warehouse' :  [1],
# _ts = YYYY-MM-DD HH:mm
'from_ts' : "2023-10-21 12:42:00.000",
'to_ts' : "2023-10-21 12:45:00.000",
}
query = f'''
SELECT *
FROM `default`.main_data_stg_t
WHERE id_forklift in %(forklit)s 
and id_warehouse in  %(warehouse)s
and event_timestamp BETWEEN %(from_ts)s and %(to_ts)s
AND (status IN ('FINISH','START')) 
ORDER by event_timestamp ASC ;
'''
result = client.execute(query, params=params)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in client.execute("DESCRIBE `default`.main_data_stg_t")])
# leave only start/finish events. assume the rover moves all the time, while not on k1 station
idle_time = (df['event_timestamp'] - df['event_timestamp'].shift(1)).sum() 
busy_time = df['event_timestamp'].max() - df['event_timestamp'].min() - idle_time

In [55]:
# Вывод пройденного расстояния за период (в разбивке по дням);
params = {
'forklit' : [0],
'warehouse' :  [1],
# _ts = YYYY-MM-DD HH:mm
'from_ts' : "2023-10-21 12:42:00.000",
'to_ts' : "2023-10-21 12:45:00.000",
}
query = f'''
SELECT *
FROM `default`.main_data_stg_t
WHERE id_forklift in %(forklit)s 
and id_warehouse in  %(warehouse)s
and event_timestamp BETWEEN %(from_ts)s and %(to_ts)s
ORDER by event_timestamp ASC ;
'''
result = client.execute(query, params=params)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in client.execute("DESCRIBE `default`.main_data_stg_t")])
df['dist'] = df.apply(lambda row: dists[row['id_point']][row['id_next_point']],axis=1)
df['dist'].sum()

155

In [22]:
# Время нахождения в каждом из статусов за период.
params = {
'forklit' : [0],
'warehouse' :  [1],
# _ts = YYYY-MM-DD HH:mm
'from_ts' : "2023-10-21 12:42:00.000",
'to_ts' : "2023-10-21 12:45:00.000",
}
query = f'''
SELECT *
FROM `default`.main_data_stg_t
WHERE id_forklift in %(forklit)s 
and id_warehouse in  %(warehouse)s
and event_timestamp BETWEEN %(from_ts)s and %(to_ts)s
ORDER by event_timestamp ASC ;
'''
result = client.execute(query, params=params)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in client.execute("DESCRIBE `default`.main_data_stg_t")])
resps = df[df['status'] != df['status'].shift()]
resps['time_in_state'] = resps['event_timestamp'] - df['event_timestamp'].shift()
resps.groupby(['status', 'id_forklift', 'id_warehouse']).aggregate({'time_in_state': 'sum' })